In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import devkit.api as dk
import pandas as pd
import datetime
import numpy as np
import gc

In [ ]:
conn = dk.MySQLProxy()
conn.connect("root", "123888", "index")

In [ ]:
query = "SELECT * FROM close \
WHERE is_st = 0 AND is_trade = 1 \
;"

In [ ]:
close = conn.query_as_dataframe("SELECT * FROM close")

In [ ]:
close = close.sort_values(['sec_id', 'date'])

In [ ]:
def cal_vol(df, date_num, cycle):
    """
    给定价格和周期计算每只股票滞后周期个交易日的波动率
    @df <dataFrame>: date<datetime.date> sec_id close
    @date <datetime.date, %Y-%m-%d>: 要求波动率的日期
    @cycle <int>: 周期 单位为天
    """
    df1 = df[df.date.le(date_num)]

    def func_vol(x, cycle):
        num = x.count()  # 当前股票在基准日之前的交易日个数
        if num < cycle:  # 如果历史交易天数小于要计算的周期 则返回空值
            return np.nan
        else:
            return x.tail(cycle).std()
    df2 = df1.groupby(['sec_id']).apply(lambda x: func_vol(x.close, cycle))
    df2.name = 'vol_{}days'.format(str(cycle))
    df3 = df2.to_frame()
    df3['date'] = date_num
    df4 = df3.dropna().reset_index()
    return df4

In [ ]:
def cal_return(df, date_num, cycle):
    """
    给定价格和周期计算每只股票滞后周期个交易日的收益率
    @df <dataFrame>: date<datetime.date> sec_id close
    @date <datetime.date, %Y-%m-%d>: 要求波动率的日期
    @cycle <int>: 周期 单位为天
    """
    df1 = df[df.date.le(date_num)]
    def func_return(x, cycle):
        num = x.count()  # 当前股票在基准日之前的交易日个数
        if num < cycle + 1:  # 如果历史交易天数小于要计算的周期 则返回空值
            return np.nan
        else:
            return (x.iloc[-1] / x.iloc[-1 * (cycle + 1)]) - 1  # 当天收盘价除以两天前的收盘价
    df2 = df1.groupby(['sec_id']).apply(lambda x: func_return(x.close, cycle))
    df2.name = 'return_{}days'.format(str(cycle))
    df3 = df2.to_frame()
    df3['date'] = date_num
    df4 = df3.dropna().reset_index()
    return df4

In [ ]:
a.tail(1)

In [ ]:
datelist = list(set(close.date.tolist()))
datelist = sorted(datelist)
for cycle in [5, 30, 90]:
    output = pd.DataFrame()
    for date in datelist:
        df = cal_vol(close.copy(), date, cycle)
        if len(df) != 0:
            output = output.append(df)
    output.to_csv(r"E:\99_daily\TODAY\formal_db\vol_{}days.csv".format(str(cycle)), index=False)

In [ ]:
datelist = list(set(close.date.tolist()))
datelist = sorted(datelist)

In [22]:
for cycle in [90]:
    output = pd.DataFrame()
    for date in datelist:
        print(date)
        df = cal_return(close.copy(), date, cycle)
        if len(df) != 0:
            output = output.append(df)
        del df
        gc.collect()
    output.to_csv(r"E:\99_daily\TODAY\formal_db\return_{}days.csv".format(
        str(cycle)), index=False)
    del output
    gc.collect()
    print("cycle=", cycle, "finished!")

2005-01-04
2005-01-05
2005-01-06
2005-01-07
2005-01-10
2005-01-11
2005-01-12
2005-01-13
2005-01-14
2005-01-17
2005-01-18
2005-01-19
2005-01-20
2005-01-21
2005-01-24
2005-01-25
2005-01-26
2005-01-27
2005-01-28
2005-01-31
2005-02-01
2005-02-02
2005-02-03
2005-02-04
2005-02-16
2005-02-17
2005-02-18
2005-02-21
2005-02-22
2005-02-23
2005-02-24
2005-02-25
2005-02-28
2005-03-01
2005-03-02
2005-03-03
2005-03-04
2005-03-07
2005-03-08
2005-03-09
2005-03-10
2005-03-11
2005-03-14
2005-03-15
2005-03-16
2005-03-17
2005-03-18
2005-03-21
2005-03-22
2005-03-23
2005-03-24
2005-03-25
2005-03-28
2005-03-29
2005-03-30
2005-03-31
2005-04-01
2005-04-04
2005-04-05
2005-04-06
2005-04-07
2005-04-08
2005-04-11
2005-04-12
2005-04-13
2005-04-14
2005-04-15
2005-04-18
2005-04-19
2005-04-20
2005-04-21
2005-04-22
2005-04-25
2005-04-26
2005-04-27
2005-04-28
2005-04-29
2005-05-09
2005-05-10
2005-05-11
2005-05-12
2005-05-13
2005-05-16
2005-05-17
2005-05-18
2005-05-19
2005-05-20
2005-05-23
2005-05-24
2005-05-25
2005-05-26

2008-01-30
2008-01-31
2008-02-01
2008-02-04
2008-02-05
2008-02-13
2008-02-14
2008-02-15
2008-02-18
2008-02-19
2008-02-20
2008-02-21
2008-02-22
2008-02-25
2008-02-26
2008-02-27
2008-02-28
2008-02-29
2008-03-03
2008-03-04
2008-03-05
2008-03-06
2008-03-07
2008-03-10
2008-03-11
2008-03-12
2008-03-13
2008-03-14
2008-03-17
2008-03-18
2008-03-19
2008-03-20
2008-03-21
2008-03-24
2008-03-25
2008-03-26
2008-03-27
2008-03-28
2008-03-31
2008-04-01
2008-04-02
2008-04-03
2008-04-07
2008-04-08
2008-04-09
2008-04-10
2008-04-11
2008-04-14
2008-04-15
2008-04-16
2008-04-17
2008-04-18
2008-04-21
2008-04-22
2008-04-23
2008-04-24
2008-04-25
2008-04-28
2008-04-29
2008-04-30
2008-05-05
2008-05-06
2008-05-07
2008-05-08
2008-05-09
2008-05-12
2008-05-13
2008-05-14
2008-05-15
2008-05-16
2008-05-19
2008-05-20
2008-05-21
2008-05-22
2008-05-23
2008-05-26
2008-05-27
2008-05-28
2008-05-29
2008-05-30
2008-06-02
2008-06-03
2008-06-04
2008-06-05
2008-06-06
2008-06-10
2008-06-11
2008-06-12
2008-06-13
2008-06-16
2008-06-17

2011-02-25
2011-02-28
2011-03-01
2011-03-02
2011-03-03
2011-03-04
2011-03-07
2011-03-08
2011-03-09
2011-03-10
2011-03-11
2011-03-14
2011-03-15
2011-03-16
2011-03-17
2011-03-18
2011-03-21
2011-03-22
2011-03-23
2011-03-24
2011-03-25
2011-03-28
2011-03-29
2011-03-30
2011-03-31
2011-04-01
2011-04-06
2011-04-07
2011-04-08
2011-04-11
2011-04-12
2011-04-13
2011-04-14
2011-04-15
2011-04-18
2011-04-19
2011-04-20
2011-04-21
2011-04-22
2011-04-25
2011-04-26
2011-04-27
2011-04-28
2011-04-29
2011-05-03
2011-05-04
2011-05-05
2011-05-06
2011-05-09
2011-05-10
2011-05-11
2011-05-12
2011-05-13
2011-05-16
2011-05-17
2011-05-18
2011-05-19
2011-05-20
2011-05-23
2011-05-24
2011-05-25
2011-05-26
2011-05-27
2011-05-30
2011-05-31
2011-06-01
2011-06-02
2011-06-03
2011-06-07
2011-06-08
2011-06-09
2011-06-10
2011-06-13
2011-06-14
2011-06-15
2011-06-16
2011-06-17
2011-06-20
2011-06-21
2011-06-22
2011-06-23
2011-06-24
2011-06-27
2011-06-28
2011-06-29
2011-06-30
2011-07-01
2011-07-04
2011-07-05
2011-07-06
2011-07-07

2014-03-25
2014-03-26
2014-03-27
2014-03-28
2014-03-31
2014-04-01
2014-04-02
2014-04-03
2014-04-04
2014-04-08
2014-04-09
2014-04-10
2014-04-11
2014-04-14
2014-04-15
2014-04-16
2014-04-17
2014-04-18
2014-04-21
2014-04-22
2014-04-23
2014-04-24
2014-04-25
2014-04-28
2014-04-29
2014-04-30
2014-05-05
2014-05-06
2014-05-07
2014-05-08
2014-05-09
2014-05-12
2014-05-13
2014-05-14
2014-05-15
2014-05-16
2014-05-19
2014-05-20
2014-05-21
2014-05-22
2014-05-23
2014-05-26
2014-05-27
2014-05-28
2014-05-29
2014-05-30
2014-06-03
2014-06-04
2014-06-05
2014-06-06
2014-06-09
2014-06-10
2014-06-11
2014-06-12
2014-06-13
2014-06-16
2014-06-17
2014-06-18
2014-06-19
2014-06-20
2014-06-23
2014-06-24
2014-06-25
2014-06-26
2014-06-27
2014-06-30
2014-07-01
2014-07-02
2014-07-03
2014-07-04
2014-07-07
2014-07-08
2014-07-09
2014-07-10
2014-07-11
2014-07-14
2014-07-15
2014-07-16
2014-07-17
2014-07-18
2014-07-21
2014-07-22
2014-07-23
2014-07-24
2014-07-25
2014-07-28
2014-07-29
2014-07-30
2014-07-31
2014-08-01
2014-08-04

2017-04-13
2017-04-14
2017-04-17
2017-04-18
2017-04-19
2017-04-20
2017-04-21
2017-04-24
2017-04-25
2017-04-26
2017-04-27
2017-04-28
2017-05-02
2017-05-03
2017-05-04
2017-05-05
2017-05-08
2017-05-09
2017-05-10
2017-05-11
2017-05-12
2017-05-15
2017-05-16
2017-05-17
2017-05-18
2017-05-19
2017-05-22
2017-05-23
2017-05-24
2017-05-25
2017-05-26
2017-05-31
2017-06-01
2017-06-02
2017-06-05
2017-06-06
2017-06-07
2017-06-08
2017-06-09
2017-06-12
2017-06-13
2017-06-14
2017-06-15
2017-06-16
2017-06-19
2017-06-20
2017-06-21
2017-06-22
2017-06-23
2017-06-26
2017-06-27
2017-06-28
2017-06-29
2017-06-30
2017-07-03
2017-07-04
2017-07-05
2017-07-06
2017-07-07
2017-07-10
2017-07-11
2017-07-12
2017-07-13
2017-07-14
2017-07-17
2017-07-18
2017-07-19
2017-07-20
2017-07-21
2017-07-24
2017-07-25
2017-07-26
2017-07-27
2017-07-28
2017-07-31
2017-08-01
2017-08-02
2017-08-03
2017-08-04
2017-08-07
2017-08-08
2017-08-09
2017-08-10
2017-08-11
2017-08-14
2017-08-15
2017-08-16
2017-08-17
2017-08-18
2017-08-21
2017-08-22

In [23]:
for cycle in [180, 360]:
    output = pd.DataFrame()
    for date in datelist:
        df = cal_return(close.copy(), date, cycle)
        if len(df) != 0:
            output = output.append(df)
        del df
        gc.collect()
    output.to_csv(r"E:\99_daily\TODAY\formal_db\return_{}days.csv".format(str(cycle)), index=False)
    del output
    gc.collect()
    print("cycle=", cycle, "finished!")

cycle= 180 finished!
cycle= 360 finished!


In [17]:
len(datelist)

3212

In [ ]:
del close
gc.collect()

In [ ]:
output.to_csv(r"E:\99_daily\TODAY\formal_db\vol_{}days.csv".format(str(cycle)), index=False)

In [ ]:
close = conn.query_as_dataframe("SELECT date, sec_id, close FROM time_series_variables WHERE is_trade=1")

In [ ]:
close['sec_id'] = close['sec_id'].apply(lambda x: x[2:] + '.' + x[:2])

In [ ]:
dk.df2mysql('root','123888','index','close',close)

In [ ]:
close.to_csv(r"E:\99_daily\TODAY\formal_db\close.csv", encoding='utf-8')

In [ ]:
ts = conn.query_as_dataframe("SELECT date, sec_id, is_trade, is_st,traded_days_until_now,stock_total_shares,stock_float_shares FROM time_series_variables WHERE YEAR(date) in (2005,2006)")

In [ ]:
ts['sec_id'] = ts['sec_id'].apply(lambda x: x[2:] + '.' + x[:2])

In [ ]:
ts['date'] = ts['date'].apply(lambda x: str(x))

In [ ]:
ic_2006=conn.query_as_dataframe("SELECT * FROM a")

In [ ]:
ic_2005=conn.query_as_dataframe("SELECT * FROM a")

In [ ]:
ic_0506 = ic_2005.append(ic_2006)

In [ ]:
ic_0506.head(3)

In [ ]:
industry = pd.read_csv(r"E:\99_daily\TODAY\industry_sw.csv")

In [ ]:
ic_0506_1.head(3)

In [ ]:
ic_0506_1 = ic_0506.merge(industry, how='inner', on=['sec_id','sec_name'])

In [ ]:
ic_0506_2 = ic_0506_1.merge(ts, how='inner', on=['date','sec_id'])

In [ ]:
ic_0506_2.head(3)

In [ ]:
ic_part1 = pd.read_csv(r"E:\99_daily\TODAY\stocks_info2.csv")
del ic_part1['Unnamed: 0']

In [ ]:
ic_part1.head(3)

In [ ]:
ic = ic_part1.append(ic_0506_2)
del ic['industry_swcode']

In [ ]:
ic.to_csv(r"E:\99_daily\TODAY\formal_db\stocks_info.csv", encoding='utf-8')

In [ ]:
ic1 = ic[ic.date == '2015-09-07']

In [ ]:
conds = (ic1.is_trade==1).multiply(ic1.is_st==0)

In [ ]:
ic2 = ic1[conds]

In [ ]:
sec_id = ic2.sec_id.tolist()

In [ ]:
sec_id

In [ ]:
conds = (ic.date=='2015-09-07').multiply(ic.is_trade==0).multiply(ic.is_st==1)

In [ ]:
ic[(ic.date=='2015-09-07').multiply(ic.is_trade==0).multiply(ic.is_st==1)]

In [ ]:
(ic.date=='2015-09-07').multiply(ic.is_trade==0)

In [ ]:
dk.df2mysql('root','123888','index','stocks_info',ic)

In [ ]:
query = "SELECT date, sec_id, close \
FROM vol_5days \
WHERE date in {} \
;".format(tuple(['2005-01-10']))

In [ ]:
test=conn.query_as_dataframe(query)

In [ ]:
close=conn.query_as_dataframe(query)

In [ ]:
close1 = close[close.close != 0]

In [ ]:
close2 = close1.sort_values(['sec_id','date'])

In [ ]:
close2['date'] = close2.date.apply(lambda x: str(x))

In [ ]:
def cal_vol(df, date_num, cycle):
    """
    给定价格和周期计算每只股票滞后周期个交易日的波动率
    @df <dataFrame>: date sec_id close
    @date <str,%Y-%m-%d>: 要求波动率的日期
    @cycle <int>: 周期 单位为天
    """
    df1 = df[df.date.le(date_num)]
    
    def func_vol(x, cycle):
        num = x.count()  # 当前股票在基准日之前的交易日个数
        if num < cycle:  # 如果历史交易天数小于要计算的周期 则返回空值
            return np.nan
        else:
            return x.tail(cycle).std()
    df2 = df1.groupby(['sec_id']).apply(lambda x: func_vol(x.close, cycle))
    df2.name = 'vol_{}days'.format(str(cycle))
    df3 = df2.to_frame()
    df3['date'] = date_num
    df4 = df3.dropna().reset_index()
    return df4

In [ ]:
datelist = list(set(close2.date.tolist()))
datelist = sorted(datelist)

In [ ]:
output = pd.DataFrame()
for date in datelist:
    df = cal_vol(close2.copy(), date, 5)
    if len(df) != 0:
        output = output.append(df)

In [ ]:
output.to_csv(r"E:\99_daily\TODAY\vol_5days.csv", encoding='utf-8')

In [ ]:
cal_vol(close2.copy(), datetime.date(2005,2,18), 5).date[0]

In [ ]:
output.shape

In [ ]:
output['sec_id'] = output['sec_id'].apply(lambda x: x[2:] + '.' + x[:2])

In [ ]:
output.head(3)

In [ ]:
dk.df2mysql('root', '123888', 'index', 'vol_5days', output)

In [ ]:
a=close[close.sec_id=='SH601318']

In [ ]:
a[a.date.between(datetime.date(2016,5,9), datetime.date(2016,5,15))]

In [ ]:
a[a.date.between(datetime.date(2016,5,9), datetime.date(2016,5,15))]['close'].std()

In [ ]:
output['date'].iloc[0]

In [ ]:
cc=output[output.date==datetime.date(2016,5,13)]

In [ ]:
cc[cc.sec_id=='SH601318']

In [ ]:
cal_vol(close2.copy(), datetime.date(2007,2,1),21)

In [ ]:
query1="SELECT date, sec_id, close FROM time_series_variables WHERE date BETWEEN '2007-01-01' AND '2007-02-01';"

In [ ]:
close1=conn.query_as_dataframe(query1)

In [ ]:
close111 = close1.sort_values(['sec_id', 'date'])

In [ ]:
close1.head(3)

In [ ]:
def cal_vol(x,cycle):
    num = x.count()
    if num<cycle:
        return np.nan
    else:
        return x.tail(21).std()
            

In [ ]:
cycle=21

In [ ]:
close_5days = close2.groupby(['sec_id']).apply(lambda x: cal_vol(x.close,cycle))

In [ ]:
close_5days.loc['SZ300708']

In [ ]:
import datetime
benchmark = datetime.date(2007,1,15)

In [ ]:
benchmark

In [ ]:
close['date'] = close.date.apply(lambda x: str(x))

In [ ]:
close.date[0]

In [ ]:
close1 = close

In [ ]:
close_5days = close.groupby(['sec_id'], as_index=False).apply(lambda x: x.tail(5))

In [ ]:
close_5days.name='vol_5days'

In [ ]:
def cal_vol(df_close,date,cycle):  
    """
    @date <datetime,%Y-%m-%d>: 要求波动率的日期
    """
    df1 = df_close[df_close.date.le(benchmark)]
    def func_vol(x, cycle):
        num = x.count()  # 当前股票在基准日之前的交易日个数
        if num < cycle:  # 如果历史交易天数小于要计算的周期 则返回空值
            return np.nan
        else:
            return x.tail(cycle).std()
    df2 = df1.groupby(['sec_id']).apply(lambda x: func_vol(x.close, cycle))
    df2.name = 'vol_{}days'.format(str(cycle))
    df3 = df2.to_frame()
    df3['date'] = date
    return df3

In [ ]:
benchmark = datetime.date(2007,1,12)
cal_vol(close2.copy(), benchmark,21)

In [ ]:
fr.to_csv(r"E:\99_daily\TODAY\fr.csv")

In [ ]:
stocks_info = conn.query_as_dataframe(query)

In [ ]:
stocks_info = stocks_info.is_st==0 and stocks_info.is_trade==1 and stocks_info.industry_sw.isin(["银行","非银金融"])'

In [ ]:
stocks_info1 = stocks_info[stocks_info.is_trade==1]# 剔除停牌股

In [ ]:
stocks_info2 = stocks_info1[stocks_info1.is_st==0] # 剔除st股

In [ ]:
stocks_info3 = stocks_info2[~stocks_info2.industry_sw.isin(["银行","非银金融"])] # 剔除金融股

In [ ]:
stocks_info4 = stocks_info3[stocks_info3.traded_days_until_now>400] # 剔除次新股

In [ ]:
stocks_info4 = stocks_info4.sort_values(['date','industry_sw','stock_float_shares'], ascending=[True,True, False])

In [ ]:
stocks_info4['date'] = stocks_info4['date'].apply(lambda x: str(x))

In [ ]:
stocks_info5 = stocks_info4.groupby(['date','industry_sw'], as_index=False).apply(lambda x: x.head(30))

In [ ]:
fr = conn.query_as_dataframe("SELECT * FROM report_variables1 WHERE (Year(date_report) BETWEEN 2010 AND 2014) AND MONTH(date_report) =12")

In [ ]:
stocks_info5.to_csv(r"E:\99_daily\TODAY\stocks_info5.csv")

In [ ]:
stocks_info5 = pd.read_csv(r"E:\99_daily\TODAY\stocks_info5.csv",encoding='gbk')

In [ ]:
stocks_info5['date'] = stocks_info5['date'].apply(lambda x: x.split('/')[0]+'-'+x.split('/')[1]+'-'+x.split('/')[2])

In [ ]:
stocks_info6=stocks_info5[stocks_info5.date.between('2010-01-01','2015-12-31')]

In [ ]:
stocks_info6

In [ ]:
stocks_info6['year'] = stocks_info6.date.apply(lambda x: x.split('-')[0])

In [ ]:
stocks_info6.head(3)

In [ ]:
flag= stocks_info6[['sec_id','year']]

In [ ]:
stocks_info5[stocks_info5.date.between(start_time=)]

In [ ]:
fr1 = fr.merge(sec_ids, how='inner', left_on=['sec_id'], right_on=['sec_id'])

In [ ]:
stocks_info5.shape

In [ ]:
stocks_info4.shape

In [ ]:
a = conn.query_as_dataframe("SELECT * FROM a")

In [ ]:
a.head(3)

In [ ]:
from devkit.api import SqliteProxy
sql = dk.SqliteProxy()
sql.connect(r"E:\02_work\sw\db\index_contents\2006.db")
table_name = "A"
df2=sql.query_as_dataframe("SELECT * from {}".format(table_name))
# print(sql.describe_table(table_name))
#print(sql.query_as_dataframe("SELECT * from {}".format(table_name)))
#print(sql.describe_table(table_name))
sql.close()

In [ ]:
df = df1.append(df2)

In [ ]:
df['sec_id'] = df['sec_id'].apply(lambda x: x.split('.')[1] + x.split('.')[0])

In [ ]:
df.head(3)

In [ ]:
ic = a.append(df)

In [ ]:
b['date'] = b['date'].apply(lambda x: str(x))

In [ ]:
a.date.tolist()

In [ ]:
b = conn.query_as_dataframe("SELECT date, sec_id, is_trade, is_st, traded_days_until_now, stock_total_shares, stock_float_shares FROM time_series_variables")

In [ ]:
b.shape

In [ ]:
c=ic.merge(b, how='inner',left_on=['sec_id','date'],right_on=['sec_id','date'])

In [ ]:
c = c.sort_values(by=['date','sec_id'])

In [ ]:
c.head(3)

In [ ]:
industry=pd.read_csv(r"E:\99_daily\TODAY\industry_sw.csv")

In [ ]:
industry['sec_id'] = industry['sec_id'].apply(lambda x: x.split('.')[1] + x.split('.')[0])

In [ ]:
industry = industry.dropna()

In [ ]:
industry[industry['industry_swcode'].isnull()]

In [ ]:
industry['industry_swcode'] = industry['industry_swcode'].apply(lambda x: int(x))

In [ ]:
df5 = c.merge(industry,how='inner', left_on=['sec_id','sec_name'], right_on=['sec_id', 'sec_name'])

In [ ]:
temp=df5[2000000:]

In [ ]:
temp

In [ ]:
dk.df2mysql('root', '123888', 'indicator', 'stocks_infos', temp, index=False,  index_label=False)

In [ ]:
temp = df5[['industry_sw','industry_swcode']]

In [ ]:
temp['industry_swcode']=temp['industry_swcode'].apply(lambda x: str(x))

In [ ]:
temp =temp.drop_duplicates()

In [ ]:
temp = temp.set_index('industry_swcode')

In [ ]:
dk.dict2json(temp.to_dict()['industry_sw'],r"E:\99_daily\TODAY\code2sw.json")

In [ ]:
df5.to_csv(r"E:\99_daily\TODAY\stocks_info2.csv",encoding='utf-8')

In [ ]:
df5 = pd.read_csv(r"E:\99_daily\TODAY\stocks_info2.csv",encoding='utf-8',index_col=0)

In [ ]:
df5.head(3)

In [ ]:
df5.shape

In [ ]:
df6 =df5.dropna(how='any')

In [ ]:
df6.shape

In [ ]:
close_info = conn.query_as_dataframe("SELECT Year(date) as year, date, sec_id, close FROM time_series_variables WHERE Year(date) BETWEEN 2010 AND 2015")

In [ ]:
close_info.head(3)

In [ ]:
close_begin=close_info.groupby(['sec_id','year'], as_index=False).apply(lambda x: x[x.date == x.date.min()])

In [ ]:
close_begin = close_begin.rename(columns={'close': 'close_begin'})

In [ ]:
close_ending=close_info.groupby(['sec_id','year'], as_index=False).apply(lambda x: x[x.date == x.date.max()])

In [ ]:
close_ending = close_ending.rename(columns={'close': 'close_ending'})

In [ ]:
close = close_begin.merge(close_ending,how='inner',left_on=['year','sec_id'], right_on=['year','sec_id'])[['year','sec_id','close_begin','close_ending']]

In [ ]:
close=close.replace({0:np.nan})

In [ ]:
close = close.dropna()

In [ ]:
close['return_rate_annual'] = close['close_ending'] / close['close_begin'] -1

In [ ]:
close['year'] = close['year'].apply(lambda x: str(x))

In [ ]:
flag.iloc[0,1]

In [ ]:
close1=close.merge(flag,how='inner',left_on=['year','sec_id'],right_on=['year','sec_id'])

In [ ]:
close1.head(3)

In [ ]:
query="SELECT * \
FROM report_variables1 \
WHERE (date_report BETWEEN '2010-01-01' AND '2015-12-31') AND Month(date_report)=12 \
ORDER BY sec_id, date_report" \

In [ ]:
fr = conn.query_as_dataframe(query)

In [ ]:
fr['year'] = fr['date_report'].apply(lambda x: str(x).split('-')[0])

In [ ]:
fr.shape

In [ ]:
fr1 = fr.merge(close1,how='inner',on=['year','sec_id'])

In [ ]:
fr1.shape[0]

In [ ]:
a=fr1.isnull().sum() / fr1.shape[0] # 缺失比例

In [ ]:
a.sort_values(ascending=False)

In [ ]:
retain_vars=a[a==0].index.tolist()

In [ ]:
fr2=fr1[retain_vars]

In [ ]:
del fr2[['']]

In [ ]:
fr2.shape

In [ ]:
close1.shape

In [ ]:
df = pd.read_csv(r"E:\99_daily\TODAY\post_process.csv", encoding='gbk')

In [ ]:
df.head(3)

In [ ]:
df1 = df.merge(close[['year', 'sec_id', 'return_rate_annual']], how='inner', left_on=['year','sec_id'], right_on=['year','sec_id'])

In [ ]:
df1

In [ ]:
a=(df1.isnull().sum()==0)

In [ ]:
df2 = df1[a[a].index.tolist()]

In [ ]:
ana_vars=df2.columns.drop(['market.1','year.1','Unnamed: 2','year','stock_float_shares_avg','market'])

In [ ]:
df2 = df2[ana_vars]

In [ ]:
df2.to_csv(r"E:\99_daily\TODAY\post_process1.csv")

In [ ]:
df2=pd.read_csv(r"E:\99_daily\TODAY\post_process1.csv",encoding='gbk')

In [ ]:
df2.head(3)

In [ ]:
import numpy as np
from sklearn.decomposition import KernelPCA
from sklearn.svm import SVR
import matplotlib.pyplot as plt

In [ ]:
y=df2['return_rate_annual']

In [ ]:
len(y)

In [ ]:
X=df2.iloc[:,4:-1]

In [ ]:
X.shape

In [ ]:
kpca=KernelPCA()

In [ ]:
X = X.apply(lambda x: (x-x.mean())/x.std())

In [ ]:
kpca.fit(X)

In [ ]:
kpca.lambdas_[:10].round(2)

In [ ]:
get_we = lambda x: x / x.sum()

In [ ]:
get_we(kpca.lambdas_)[:15].sum()

In [ ]:
kpca.transform(X).iloc[:,:16]

In [ ]:
b=pd.DataFrame(kpca.transform(X))

In [ ]:
X_new=b.iloc[:,:16]

In [ ]:
comp[:16].shape

In [ ]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

In [ ]:
svr_rbf.fit(X_new,y)

In [ ]:
svr_lin = SVR(kernel='linear', C=1e3)

In [ ]:
svr_lin.fit(X,y)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
y_predict=svr_lin.predict(X)

In [ ]:
y_predict=svr_rbf.predict(X_new)

In [ ]:
r2_score(y, y_predict)

In [ ]:
r2_score(y, y_predict)

In [ ]:
r2_score(y, y_predict)

In [ ]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(X, y).predict(X)
y_lin = svr_lin.fit(X, y).predict(X)
y_poly = svr_poly.fit(X, y).predict(X)

In [ ]:
lw = 2
plt.scatter(X, y, color='darkorange', label='data')
plt.hold('on')
# plt.plot(X, y_lin, color='c', lw=lw, label='Linear model')
plt.plot(X, y_rbf, color='navy', lw=lw, label='RBF model')
# plt.plot(X, y_poly, color='cornflowerblue', lw=lw, label='Polynomial model')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')
plt.legend()
plt.show()

In [ ]:
cols_to_retain

In [ ]:
close_info['year'] = close_info['date'].apply(lambda x: datetime.datetime.year(x))

In [ ]:
flag = conn.query_as_dataframe("SELECT Year(date) as year, sec_id, AVG(stock_total_shares) as stock_float_shares_avg,FROM time_series_variables GROUP BY Year(date), sec_id;")

In [ ]:
fr = conn.query_as_dataframe("SELECT *, YEAR(date_report) as year FROM report_variables1 WHERE (Year(date_report) BETWEEN '2005' and '2015') and  MONTH(date_report) = 12;")

In [ ]:
df = fr.merge(flag, how='inner',right_on=['sec_id','year'],left_on=['sec_id','year'])

In [ ]:
df['market'] = df['sec_id'].apply(lambda x: "上交所" if x[:2]=='SH' else "深交所")

In [ ]:
df

In [ ]:
df=df.sort_values(by=['year','market','stock_float_shares_avg'], ascending=[True, True,False])

In [ ]:
df1=df.groupby(by=['year','market']).apply(lambda x: x.head(150))

In [ ]:
cols_to_retain=(df1.isnull().sum()/df1.shape[0] < 0.05).index.tolist()

In [ ]:
df2=df1.copy()

In [ ]:
ana_vars=df2.columns.drop(['sec_id','date_report','date_published','year','stock_float_shares_avg','market'])

In [ ]:
df2 = df1[ana_vars]

In [ ]:
df2[ana_vars] = (df2[ana_vars] - df2[ana_vars].min())/ (df2[ana_vars].max() - df2[ana_vars].min())

In [ ]:
df2.to_csv(r"C:\Users\youlin\Desktop\post_process.csv")

In [ ]:
flag = conn.query_as_dataframe("SELECT Year(date) as year, sec_id, AVG(stock_total_shares) as stock_float_shares_avg,FROM time_series_variables GROUP BY Year(date), sec_id;")

In [ ]:
output=pd.DataFrame()
for year in df.year.drop_duplicates().tolist():
    output=output.append(df[df.year.isin([year])].head(150))

In [ ]:
output.shape

In [ ]:
df.shape

In [ ]:
4

A2B3D

A4C2E

A5D

C3B

In [ ]:
all={'A':{'B':0,'C':0,'D':0,'E':0},'B':{'A':0,'C':0,'D':0,'E':0},'C':{'B':0,'A':0,'D':0,'E':0},'D':{'B':0,'C':0,'A':0,'E':0},'E':{'B':0,'C':0,'D':0,'A':0}}

In [ ]:
all

In [ ]:
for str in ['A2B3D','A4C2E']:
    alpha = re.findall(r'([a-zA-Z]+)',str)
    dist =  re.findall(r'([0-9]+)',str)
    for index,value in enumerate(alpha):
        if index > len(alpha) -2 :
            break
        all[value][alpha[index+1]]=dist[index]

In [ ]:
import copy
all2=copy.deepcopy(all)

In [ ]:
for key1 in all:
    for key2 in all[key1]:
        if all[key1][key2] ==0:
            del all2[key1][key2]

In [ ]:
all2

In [ ]:
a={'a':1,'b':2}

In [ ]:
del a['a']

In [ ]:
a

In [ ]:
for key in all:
    for sub_key in all[key]:
        if sub_key in all:
            for sub_sub_key in all[sub_key]
            max_root[key]=all[key][sub_key]+

In [ ]:
a=re.findall('[0-9]*','s2d23e')

In [ ]:
def search(site):
    for sub in all2[site]:
        if sub in all2:
            root=search(site)
        else:
            root={site:sub}

In [ ]:
search('A')

In [ ]:
 re.findall(r'([0-9]+)','A4C2E')

In [ ]:
 re.findall(r'([a-zA-Z]+)','A4C2E')

In [ ]:
df = pd.DataFrame({'Sp':['a','b','c','d','e','f'], 'Mt':['s1', 's1', 's2','s2','s2','s3'], 'Value':[1,2,3,4,5,6], 'Count':[3,2,5,10,10,6]})

In [ ]:
df

In [ ]:
df.groupby(by=['Mt']).apply(lambda x: x[x.Count==x.Count.max()])

In [ ]:
df.groupby(by=['Mt']).agg(max)

In [ ]:
df

In [ ]:
df.groupby(by=['Mt'])['Count'].transform(max)

In [ ]:
idx=df.groupby(by=['Mt'])['Count'].transform(max)

In [ ]:
idx

In [ ]:
df['Count'] == idx